In [1]:
## 환경
from dotenv import load_dotenv
import warnings
import os

warnings.filterwarnings("ignore")
load_dotenv()

True

In [ ]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다. 추적 안 하고싶으면 set_enalbel =Fasle로 설정
logging.langsmith("wiset-final")

In [3]:
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine, inspect, text
import os

db_path = os.getenv("DB_PATH")
if not os.path.exists(db_path):
    raise Exception("데이터베이스 파일이 존재하지 않습니다.")

db_uri = f"sqlite:///{db_path}"

# 엔진 생성
engine = create_engine(db_uri)

# 엔진 생성
engine = create_engine(db_uri)

# SQLDatabase 인스턴스 생성 시 엔진 객체 전달
db = SQLDatabase(engine)

In [4]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate

# 사용 가능한 테이블 이름 출력
print(db.get_usable_table_names())

print(db.dialect)

['Categories', 'CustomerCustomerDemo', 'CustomerDemographics', 'Customers', 'EmployeeTerritories', 'Employees', 'Order Details', 'Orders', 'Products', 'Regions', 'Shippers', 'Suppliers', 'Territories']
sqlite


In [5]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Based on the table schema below, write a SQL query that wold answer the user's question:
{schema}

Question: {question}
SQL Query
"""

prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['question', 'schema'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'schema'], template="\nBased on the table schema below, write a SQL query that wold answer the user's question:\n{schema}\n\nQuestion: {question}\nSQL Query\n"))]


In [6]:
def get_schema(_):
    return db.get_table_info()

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [8]:
llm = ChatOpenAI()

sql_chain =(
    RunnablePassthrough.assign(schema=get_schema)
    | prompt 
    | llm.bind(stop="\nSQL Result:")
    | StrOutputParser()
)

In [9]:
sql_chain.invoke({"question": "How may Employees are there?"})


'SELECT COUNT(EmployeeID) AS TotalEmployees\nFROM Employees;'

In [10]:
template = """
Based on the table schema below, qustion, sql query and sql response, write a natural language response that would answer the user's question:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt = ChatPromptTemplate.from_template(template)


In [11]:
def run_query(query):
    return db.run(query)

In [12]:
run_query("SELECT COUNT(*) AS TotalEmployees FROM Employees;")

'[(9,)]'

In [13]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema = get_schema,
        #response = lambda variables: print(variables)
        response = lambda variables: run_query(variables["query"])
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
full_chain.invoke({"question": "How many Employees are there?"})

'There are a total of 9 employees in the company.'